In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11838
2,Huelva,Confirmados PDIA 14 días,741
3,Huelva,Tasa PDIA 14 días,"145,08275199072725"
4,Huelva,Confirmados PDIA 7 días,435
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,20
629,Municipio de Huelva sin especificar,Total Confirmados,156
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11838.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3371.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1131 personas en los últimos 7 días 

Un positivo PCR cada 622 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11838.0,435.0,741.0,510743.0,85.170037,145.082752,48.0
Huelva-Costa,6756.0,293.0,487.0,288115.0,101.695504,169.029728,30.0
Huelva (capital),3371.0,127.0,231.0,143663.0,88.401328,160.792967,15.0
Condado-Campiña,3585.0,95.0,168.0,155057.0,61.267792,108.347253,10.0
Sierra de Huelva-Andévalo Central,1342.0,27.0,56.0,67571.0,39.957970,82.875790,7.0
Isla Cristina,552.0,83.0,103.0,21264.0,390.331076,484.386757,5.0
Aljaraque,458.0,13.0,19.0,21260.0,61.147695,89.369708,4.0
Cartaya,451.0,19.0,34.0,19974.0,95.123661,170.221288,2.0
Lepe,1005.0,20.0,40.0,27431.0,72.910211,145.820422,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),58.0,6.0,20.0,2364.0,253.807107,846.023689,1.0
Almendro (El),21.0,5.0,5.0,826.0,605.326877,605.326877,0.0
Cumbres Mayores,19.0,2.0,10.0,1755.0,113.960114,569.800570,0.0
Isla Cristina,552.0,83.0,103.0,21264.0,390.331076,484.386757,5.0
Villablanca,69.0,2.0,12.0,2848.0,70.224719,421.348315,0.0
Palma del Condado (La),581.0,20.0,39.0,10761.0,185.856333,362.419849,0.0
Lucena del Puerto,112.0,1.0,8.0,3371.0,29.664788,237.318303,1.0
Moguer,423.0,32.0,43.0,22088.0,144.875045,194.675842,1.0
Corteconcepción,19.0,1.0,1.0,548.0,182.481752,182.481752,NaN


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,112.0,1.0,8.0,3371.0,29.664788,237.318303,1.0,0.125000
Villablanca,69.0,2.0,12.0,2848.0,70.224719,421.348315,0.0,0.166667
Cumbres Mayores,19.0,2.0,10.0,1755.0,113.960114,569.800570,0.0,0.200000
Almonte,454.0,3.0,10.0,24191.0,12.401306,41.337688,1.0,0.300000
Cerro de Andévalo (El),58.0,6.0,20.0,2364.0,253.807107,846.023689,1.0,0.300000
Punta Umbría,246.0,5.0,11.0,15242.0,32.804094,72.169007,0.0,0.454545
Sierra de Huelva-Andévalo Central,1342.0,27.0,56.0,67571.0,39.957970,82.875790,7.0,0.482143
Gibraleón,352.0,1.0,2.0,12607.0,7.932101,15.864202,0.0,0.500000
Lepe,1005.0,20.0,40.0,27431.0,72.910211,145.820422,2.0,0.500000
